## Load the pickle

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
%%time

# import pickle
with open("Random_Forest_20.pkl",'rb') as file:
    m = pickle.load(file)


CPU times: total: 0 ns
Wall time: 4.02 ms


In [10]:
%%time
model = m['model']
feature_names = m['feature_names']
label_encoder = m['label_encoder']

CPU times: total: 0 ns
Wall time: 0 ns


Extract Rules

In [11]:
%%time

import pandas as pd

from sklearn.tree import _tree
import numpy as np

def get_rules(tree, feature_names, class_names):
    try:
        tree_ = tree.tree_
        feature_name = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in tree_.feature
        ]

        paths = []
        path = []

        def recurse(node, path, paths):
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_name[node]
                threshold = tree_.threshold[node]
                p1, p2 = list(path), list(path)
                p1 += [f"({name} <= {np.round(threshold, 3)})"]
                recurse(tree_.children_left[node], p1, paths)
                p2 += [f"({name} > {np.round(threshold, 3)})"]
                recurse(tree_.children_right[node], p2, paths)
            else:
                path += [(tree_.value[node], tree_.n_node_samples[node])]
                paths += [path]

        recurse(0, path, paths)

        samples_count = [p[-1][1] for p in paths]
        ii = list(np.argsort(samples_count))
        paths = [paths[i] for i in reversed(ii)]

        rules = []
        for path in paths:
            rule = "if "

            for p in path[:-1]:
                if rule != "if ":
                    rule += " and "
                rule += str(p)
            
            if class_names is None:
                rule += ": return " + str(np.round(path[-1][0][0][0], 3))
            else:
                classes = path[-1][0][0]
                l = np.argmax(classes)
                rule += f"class: {class_names[l]} (proba: {np.round(100.0 * classes[l] / np.sum(classes), 2)}%)"

            rules += [rule]

        return rules
    except (AttributeError, NameError) as e:
        print(f"Error extracting rules: {e}")
        return None


def get_all_rules(forest, feature_names, class_names):
    all_rules = []
    for tree in forest.estimators_:
        rules = get_rules(tree, feature_names, class_names)
        if rules:
            all_rules.append(rules)
    return all_rules




  
class_names = None  

# Extract rules from the Random Forest
all_rules = get_all_rules(model, feature_names, class_names)

# Print the extracted rules
# for i, tree_rules in enumerate(all_rules):
#     print(f"\nRules from Tree {i+1}")
#     for rule in tree_rules:
#         print(rule)

# Write rules to a file if needed
with open('output.txt', 'w') as file:
    for i, tree_rules in enumerate(all_rules):
        file.write(f"\nRules from Tree {i+1}\n")
        for rule in tree_rules:
            file.write(rule + '\n')

print("Rules have been extracted and written to",file.name)
    



Rules have been extracted and written to output.txt
CPU times: total: 906 ms
Wall time: 5.79 s


Read Parameters

In [12]:
import re
import pickle

def extract_words_after_parentheses(pseudocode):
    
    matches = re.findall(r'\(\s*([a-zA-Z]+)\s*>', pseudocode)
    
    extracted_set = set(matches)
    
    return extracted_set

def read_pseudocode_from_file(file_path):
    with open(file_path, 'r') as file:
        pseudocode = file.read()
    return pseudocode

file_path = "output.txt"
pseudocode = read_pseudocode_from_file(file_path)

result_set = extract_words_after_parentheses(pseudocode)

print(result_set)



def export_set(my_set, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(my_set, file)

export_file_path = 'exported_set.pkl'

export_set(result_set, export_file_path)


{'vinp', 'pd', 'vdd', 'temperature', 'process'}


Generate the Function

In [13]:

import pickle

def import_set(file_path):
    with open(file_path, 'rb') as file:
        my_set = pickle.load(file)
    return my_set

import_file_path = 'exported_set.pkl'


def generate_python_code_from_pseudocode(pseudocode_lines, input_parameters):
    python_code = ""
    rule_count = 1
    capture_lines = False

    for line in pseudocode_lines:
        if f"Rules from Tree {rule_count}" in line:
            parameter_string = ', '.join(f"{param}=None" for param in input_parameters)
            python_code += f"\ndef decision_tree_rule_{rule_count}({parameter_string}):" + "\n"
            rule_count += 1
            capture_lines = True

        elif capture_lines and line.strip():  
            indented_code_block = "    " + line.replace('return', '\n        return')
            python_code += f"{indented_code_block}" + "\n"

    return python_code


input_parameters = import_set(import_file_path)
print("Imported Set:", input_parameters)

file_path = "output.txt" 
with open(file_path, 'r') as file:
    pseudocode_lines = file.readlines()


python_code = generate_python_code_from_pseudocode(pseudocode_lines, input_parameters)
print("Python Code generated")

output_file_path = "TreeFunction.py"  
with open(output_file_path, 'w') as code_file:
    code_file.write(python_code)

print(f"Python code has been saved to: {output_file_path}")


Imported Set: {'vinp', 'pd', 'vdd', 'temperature', 'process'}
Python Code generated
Python code has been saved to: TreeFunction.py


Prediction

In [15]:
%%time
import numpy as np
import pickle

def import_set(file_path):
    with open(file_path, 'rb') as file:
        my_set = pickle.load(file)
    return my_set

import_file_path = 'exported_set.pkl'




def evaluate_all_conditions(**kwargs):
    results = []

    function_file_path = "TreeFunction.py"

    with open(function_file_path, 'r') as file:
        functions_code = file.read()

    for i in range(1,21):
        function_name = f"decision_tree_rule_{i}"
        exec(functions_code)
        function = locals()[function_name]
        result = function(**kwargs) or 0
        results.append(result)

    return results

with open('Random_Forest_20.pkl','rb') as f:
    m = pickle.load(f)

model = m['model']
feature_names = m['feature_names']
label_encoder = m['label_encoder']

input_parameters = import_set(import_file_path)

print("Imported Set:", input_parameters)

## input  values
new_data = [[3.0000, 3.0000, 1.638227, 85, 'slownslowp']]
new_df = pd.DataFrame(new_data, columns=['vdd', 'pd', 'vinp', 'temperature', 'process'])
new_df['process'] = label_encoder.transform(new_df['process'])

vinp_value = new_df['vinp'][0]
pd_value = new_df['pd'][0]
vdd_value = new_df['vdd'][0]
process_value = new_df['process'][0]
temperature_value =new_df['temperature'][0]


input_values = {'vinp': vinp_value, 'pd': pd_value, 'vdd': vdd_value, 'process':process_value,'temperature':temperature_value}

result_list = evaluate_all_conditions(**input_values)

print("Length:", len(result_list))
# print("Results:", result_list)
print("Prediction:", np.average(result_list))


Imported Set: {'vinp', 'pd', 'vdd', 'temperature', 'process'}
Length: 20
Prediction: 0.76765
CPU times: total: 22.1 s
Wall time: 22.6 s


In [20]:
%%time

import matplotlib.pyplot as plt
import pandas as pd
from itertools import count
from matplotlib.animation import FuncAnimation
import pickle
import numpy as np

def import_set(file_path):
    with open(file_path, 'rb') as file:
        my_set = pickle.load(file)
    return my_set

def evaluate_all_conditions(**kwargs):
    results = []

    function_file_path = r"TreeFunction.py"

    with open(function_file_path, 'r') as file:
        functions_code = file.read()

    for i in range(1, 21):
        function_name = f"decision_tree_rule_{i}"
        exec(functions_code)
        function = locals()[function_name]
        result = function(**kwargs) or 0
        results.append(result)

    return results

data = pd.read_csv(r"C:\Users\Admin\Desktop\forthSem\test\fastnfastp_3.6V_45.csv")

x_data_input = []
pdd = []
vdd = []
vinp = []
temperature = []
process = []

index = count()

def calculate_metrics(y_data, vinn):
    rmse = np.sqrt(((y_data - vinn) ** 2).mean())
    mae = np.abs(y_data - vinn).mean()
    noise = y_data - vinn
    signal_power = np.mean(y_data ** 2)
    noise_power = np.mean(noise ** 2)
    snr = 10 * np.log10(signal_power / noise_power)
    return rmse, mae, snr

def animate_input(i):
    if next(index) < len(data):
        x = data['time'].iloc[next(index)]
        p = data['pd'].iloc[next(index)]
        vin = data['vinp'].iloc[next(index)]
        vd = data['vdd'].iloc[next(index)]
        temp = data['temperature'].iloc[next(index)]
        pro = data['process'].iloc[next(index)]

        x_data_input.append(x)
        pdd.append(p)
        vinp.append(vin)
        vdd.append(vd)
        temperature.append(temp)
        process.append(pro)
        
        ax1.clear()
        ax1.plot(x_data_input, pdd, label='pdd')
        ax1.plot(x_data_input, vinp, label='vinp')
        ax1.plot(x_data_input, vdd, label='vdd')
        ax1.plot(x_data_input, temperature, label='temperature')
        ax1.plot(x_data_input, process, label='process')
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Voltage')
        ax1.set_title('Real-time Waveform Plot (Input)')
        ax1.legend()

x_data_output = []
y_data = []
vinn = []
rmse_values = []
mae_values = []
snr_values = []

def animate_output(i):
    if next(index) < len(data):
        x = data['time'].iloc[next(index)]
        y = data['vinn'].iloc[next(index)]
        vin = prediction[next(index)]
        
        x_data_output.append(x)
        y_data.append(y)
        vinn.append(vin)
        
        rmse, mae, snr = calculate_metrics(np.array(y_data), np.array(vinn))
        rmse_values.append(rmse)
        mae_values.append(mae)
        snr_values.append(snr)
        
        ax2.clear()
        ax2.plot(x_data_output, y_data, label='Actual', color='b')
        ax2.plot(x_data_output, vinn, label='Predicted', color='red')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Voltage')
        ax2.set_title('Real-time Waveform Plot (Output) \n(RMSE: {:.4f}, MAE: {:.4f}, SNR: {:.4f})'.format(rmse, mae, snr))
        ax2.legend()

with open('Random_Forest_20.pkl', 'rb') as file:
    model_data = pickle.load(file)
 
loaded_model = model_data['model']
feature_names = model_data['feature_names']
label_encoder = model_data['label_encoder']

new_data = data[['vdd', 'pd', 'vinp', 'temperature','process']]
new_df = pd.DataFrame(new_data, columns=['vdd', 'pd', 'vinp', 'temperature','process'])
new_df['process'] = label_encoder.transform(new_df['process'])
prediction = []

for index, row in new_df.iterrows():
    input_values = row.to_dict()  
    result_list = evaluate_all_conditions(**input_values)
    prediction.append(result_list)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,10))
ax1.set_title('Title 1', pad=20)  
ax2.set_title('Title 2', pad=20)
ax1.set_xlabel('X-axis label', labelpad=10) 
ax1.set_ylabel('Y-axis label', labelpad=10) 
ax2.set_xlabel('X-axis label', labelpad=10) 
ax2.set_ylabel('Y-axis label', labelpad=10)

ani_input = FuncAnimation(fig, animate_input, interval=50, cache_frame_data=False)
ani_output = FuncAnimation(fig, animate_output, interval=50, cache_frame_data=False)

plt.tight_layout()
plt.show()
print("Graphs plotted")


KeyboardInterrupt: 